1. How do you debug a pod stuck in CrashLoopBackoff?

In [ ]:
--> “If a pod is stuck in CrashLoopBackOff, I usually follow a structured debugging process. First, I check the pod’s status with kubectl get pods to confirm the issue.
--> Then I run kubectl describe pod <pod-name> to see events, like whether it’s failing due to liveness/readiness probes, OOMKilled, or some other scheduling issue.
--> Next, I check the container logs using kubectl logs <pod-name>, and if the pod is restarting too quickly, I use the --previous flag to capture logs from the previous crash. [kubectl logs <pod-name> --previous
] if pod as multiple container [kubectl logs <pod-name> -c <container-name> --previous]
Based on experience, common causes are application errors, misconfigured environment variables, wrong commands/entrypoints in the image, failed probes, or resource constraints. For example, if the logs show ‘OOMKilled’, I would increase memory limits. If the liveness probe is failing, I would adjust the probe configuration or fix the app’s startup.
If logs don’t help, I use kubectl debug or override the entrypoint with a sleep command so I can exec into the container and inspect configs or dependencies.
So my approach is: check describe, check logs, validate probes and resource limits, and then fix the underlying application or configuration issue before redeploying.”*

In [ ]:
2.How will you stop pod in k8s?

In [ ]:
--> *“In Kubernetes, we normally don’t stop a pod directly, because pods are managed by higher-level controllers like Deployments, ReplicaSets, or StatefulSets. If I just delete the pod using kubectl delete pod <pod-name>, the controller will immediately recreate it to maintain the desired replicas.
If I really want to stop the pod permanently, 

--> I need to scale down the controller (for example, kubectl scale deployment <name> --replicas=0). That way, Kubernetes won’t recreate the pod.
cmd: kubectl scale deployment sindhu-deployment  --replicas=0 -n default 

--> If it’s a standalone pod without a controller, deleting it with kubectl delete pod will stop it.

--> For debugging, I can also temporarily ‘stop’ a pod by running kubectl drain on the node, or by setting replicas=0 in the YAML manifest.”*

In [ ]:
3. how will you replicate a pod ?

In [ ]:
--> *“In Kubernetes, we don’t usually replicate a single pod manually because pods are ephemeral and managed by controllers. Instead, we use controllers like Deployments, ReplicaSets, or StatefulSets to handle replication automatically.
    For example, if I have a pod defined in a Deployment, I can increase the number of replicas by updating the replicas field:*

--> kubectl scale deployment <deployment-name> --replicas=3

--> *This ensures Kubernetes maintains 3 copies of the pod. If a pod crashes, the controller automatically recreates it.

--> For standalone pods (not managed by a controller), I would need to manually create additional pods with the same specification, but that’s not recommended in production.”*

In [ ]:
4. command to get logs in k8s?

In [ ]:
“To check logs in Kubernetes, I use kubectl logs <pod-name>.
 If the pod has multiple containers, I specify the container with -c <container-name>. 
 For pods that are crashing repeatedly, I use the --previous flag to see logs from the last crashed instance.
 And if the pod is in a different namespace, I add -n <namespace>.”

In [ ]:
5.what will you do if pod is getting more load and we need to stay it healthy before it gets died?

In [ ]:
“If a pod is experiencing high load and we want to keep it healthy, I would handle it in multiple ways:

Horizontal Scaling: Increase the number of pod replicas using a Deployment or ReplicaSet. For example:*

kubectl scale deployment <deployment-name> --replicas=3

Or set up a Horizontal Pod Autoscaler (HPA) to automatically scale based on CPU/memory load:

kubectl autoscale deployment <deployment-name> --cpu-percent=50 --min=2 --max=10

Vertical Scaling: Increase the resources (CPU/memory) allocated to the pod in its YAML spec to handle higher load.

Load Distribution: Ensure the service is properly load-balanced across the pods.

Readiness/Liveness Probes: Check that probes are configured correctly so Kubernetes can restart unhealthy pods before they fail completely.

So my approach is proactive: scale horizontally, ensure sufficient resources, and configure proper probes to maintain pod health under high load.”

In [ ]:
6. How do you manage multiple environments (dev, stage, prod)?

Separate namespaces or clusters.

Use Helm or Kustomize for environment-specific configs.

In [ ]:
"Auto-generated Terraform code is useful to bootstrap,
 but it’s not production-ready. It usually contains hardcoded values,
  lacks variables and modules, and doesn’t follow best practices like state management or secrets handling.
   We should refactor it into reusable modules with variables, 
  outputs, and remote backends before using it for real infrastructure."

In [ ]:
7. what is kubernetes, and what problem does it solve?

In [ ]:
Kubernetes, or K8s, is an open-source container orchestration platform used to deploy,
manage, and scale containerized applications. It solves the challenges of running applications at scale, 
where managing hundreds of containers manually becomes complex. Kubernetes automates deployments, 
rollouts, and rollbacks. It ensures high availability by rescheduling workloads if a container or node 
fails. It also handles service discovery, load balancing, and auto-scaling based on demand. 
Additionally, it optimizes resource usage by scheduling workloads efficiently across nodes. 
In short, Kubernetes makes running applications in production more reliable and automated.

In [ ]:
8.what is container orchestration platform why not other why k8s?

In [ ]:
A container orchestration platform automates how containers are deployed, scaled, and managed across multiple servers.
While there are tools like Docker Swarm, Kubernetes is preferred because it’s more mature, cloud-agnostic, 
and has strong community and ecosystem support. It offers advanced features like self-healing,
auto-scaling, and rolling updates, which make it production-ready. 
That’s why Kubernetes has become the industry standard.

In [ ]:
9.Explain the architecture of kubernetes?

In [ ]:
Kubernetes has a master–worker architecture. The control plane (master) manages the cluster and has components 
like the API server (entry point), etcd (stores cluster state), controller manager (maintains desired state),
and scheduler (assigns pods to nodes). The worker nodes run the actual applications.
Each node has a kubelet (communicates with master), kube-proxy (handles networking), 
and a container runtime like Docker or containerd to run containers. I
n simple terms, the master decides what should happen, and the workers actually run the workloads.

In [ ]:
10.what is pod in kubernetes?

In [ ]:
A Pod in Kubernetes is the smallest deployable unit, which represents one or more containers 
that are deployed together on the same node. Containers in a Pod share the same network namespace,
IP address, and storage volumes, which allows them to communicate easily. Pods are the basic building blocks of Kubernetes, 
and higher-level objects like Deployments or ReplicaSets manage Pods for scaling and reliability.

In [ ]:
11. how does kubernetes ensure high avilability and scalability of applications?

In [ ]:
Kubernetes ensures high availability by running applications in multiple replicas using ReplicaSets/Deployments. 
If a pod or even a node fails, the control plane automatically reschedules the workload on a healthy node,
so the application stays available. It also supports load balancing through Services, 
which distribute traffic across pods.For scalability, Kubernetes has 
Horizontal Pod Autoscaler (HPA) and Vertical Pod Autoscaler (VPA).
HPA automatically increases or decreases the number of pods based on metrics like CPU or memory usage, 
while VPA adjusts resource limits. It can also scale across nodes using Cluster Autoscaler in cloud environments.
In short, Kubernetes keeps applications running reliably and scales them up or down automatically based on demand

In [ ]:
12.what is kubernetes service, how does it work?

In [ ]:
A Kubernetes Service is an abstraction that defines a logical set of pods and a policy to access them.
Since pods are ephemeral (they can die and get recreated with new IPs), a Service provides a stable IP address and DNS name so other components or users can reliably communicate with the pods.
How it works:when you create a Service, Kubernetes assigns it a cluster IP.

It automatically maps the service to the backend pods using labels and selectors.

kube-proxy on each node manages traffic routing from the Service IP to the pod IPs.

Services can be of different types:

ClusterIP – accessible only within the cluster.

NodePort – exposes the service on a static port of each node.

LoadBalancer – integrates with cloud provider load balancers.

ExternalName – maps the service to an external DNS name.

In short, Services make pods discoverable and allow stable access despite pods being dynamic.

In [ ]:
13. difference between deployments and statefulsets?

In [ ]:
Both Deployments and StatefulSets are used to manage pods, but they are designed for different use cases.
Deployments:
Used for stateless applications like web servers or APIs.
Pods are interchangeable, meaning any pod can replace another.
Supports rolling updates and rollbacks easily.
Pod names are random and IPs are dynamic.
StatefulSets:
Used for stateful applications like databases (MySQL, Cassandra) that need persistent storage.
Pods are unique and ordered; they maintain a stable identity (persistent hostname and storage).
Supports ordered deployment, scaling, and deletion.
Each pod gets a stable network identity and persistent volume.
In short: Use Deployments for stateless apps, and StatefulSets for stateful apps that need stable storage and identity.

In [ ]:
14. Explain the concept of kubernetes Persistent Volumes(pvs) and persistent Volume Claims(Pvcs)?

In [ ]:
In Kubernetes, pods are ephemeral, meaning their storage is lost if the pod dies.
To manage persistent storage, Kubernetes uses Persistent Volumes (PVs) and Persistent Volume Claims (PVCs).
Persistent Volume (PV): A PV is a piece of storage in the cluster provisioned by an admin 
or dynamically by a storage class. It abstracts the underlying storage 
(like NFS, AWS EBS, GCE Persistent Disk, etc.).
Persistent Volume Claim (PVC): A PVC is a request for storage by a pod. It specifies size, access mode,
 and storage class. Kubernetes then binds the PVC to a suitable PV.
How it works:
Admin creates PVs (or uses dynamic provisioning).
Developer creates a PVC in the pod spec.
Kubernetes finds a matching PV and binds it.
The pod can now read/write to persistent storage, even if it is rescheduled to another node.
In short: PV is the actual storage, PVC is the request for storage, and Kubernetes handles the binding between
them to provide persistent storage to pods.

In [ ]:
15.what are kubernetes Lables and selectors, and how they used?

In [ ]:
Labels in Kubernetes are key-value pairs attached to objects like pods, deployments, or services.
They are used to organize, identify, and group resources. Labels themselves don’t provide behavior; 
they are just metadata.Selectors are used to query or select objects based on their labels. 
Kubernetes uses selectors to determine which pods a Service, Deployment, or other resource should manage.
How they are used:
Grouping pods for a Service: A Service uses a label selector to route traffic to matching pods.
Managing workloads: Deployments use label selectors to manage the pods they create.
Filtering and monitoring: Labels help with kubectl queries, monitoring, and organizing resources.

In [ ]:
16. Explain Kubernetes Rolling Updates and Rollbacks?

In [ ]:
A Rolling Update in Kubernetes is a way to update applications gradually without downtime.
Instead of stopping all pods at once, Kubernetes updates pods one by one (or in batches), 
ensuring that some instances of the application are always available. This is typically done via a Deployment, 
which manages the process automatically.If something goes wrong during the update, Kubernetes allows 
a Rollback, which reverts the Deployment to a previous stable version. Rollbacks use the Deployment’s 
revision history, so you can quickly restore the application to its last working state.
Key points:
Rolling updates provide zero downtime during upgrades.
Rollbacks ensure safety if the new version fails.
Controlled via Deployment strategy, with options like maxUnavailable and maxSurge to control the speed and safety of the rollout.
In short: Rolling updates let you update applications smoothly, and rollbacks let you revert if anything goes wrong.

In [ ]:
17. what is Kubernetes configmap, and how is it used?

In [ ]:
A ConfigMap in Kubernetes is an object used to store non-confidential configuration data in key-value pairs. It allows you to decouple configuration from container images, so you can change application settings without rebuilding the image.
How it’s used:
You can inject ConfigMap data into pods as environment variables or mount it as files inside the container.
ConfigMaps are useful for application settings, config files, or command-line arguments.
This enables dynamic configuration; you can update the ConfigMap, and pods can reload the configuration without redeploying.

In [ ]:
18. What is the purpose of Kubernetes ingress?

In [ ]:
A Kubernetes Ingress is an API object that manages external access to services in a cluster, 
typically HTTP or HTTPS traffic. It provides a single entry point for multiple services, 
allowing you to route requests based on hostnames or URL paths.
Purpose and features:
Routing traffic: Directs incoming requests to the correct service based on rules.
TLS/SSL termination: Can handle HTTPS connections at the ingress level.
Load balancing: Distributes traffic to backend pods.
Centralized access management: Makes exposing multiple services simpler than using multiple NodePorts or LoadBalancers.

In [ ]:
19.how does kubernetes handle container networking?

In [ ]:
Kubernetes handles container networking using a flat network model where every pod gets its own IP address,
and all pods can communicate with each other without NAT.
This simplifies networking and allows pods to talk to each other directly.
Key points of Kubernetes networking:
Pod-to-Pod communication: Every pod gets a unique IP. Containers within a pod share the same network namespace.
Service-to-Pod communication: Kubernetes Services provide a stable IP and DNS name to access pods, even if pods are recreated.
kube-proxy: Runs on each node to manage routing and load balancing from services to pods.
Network plugins (CNI): Kubernetes uses the Container Network Interface (CNI) standard,
and plugins like Calico, Flannel, or Weave handle the actual networking implementation.

In [ ]:
20. Explain kubernetes RBAC and its importance?

In [ ]:
RBAC (Role-Based Access Control) in Kubernetes is a way to control who can do what within the cluster. 
It lets you define roles with specific permissions and then bind them to users, groups, or service accounts.
Key components:
Role / ClusterRole: Defines a set of permissions (like get, list, create) on resources. Role is namespace-scoped, while ClusterRole is cluster-wide.
RoleBinding / ClusterRoleBinding: Assigns a Role or ClusterRole to a user, group, or service account.
Importance:
Ensures security by granting only the required access.
Helps follow the principle of least privilege.
Allows multi-tenant clusters safely, where different teams or applications have restricted access.
